In [1]:
import os
import sys
os.chdir('../')
sys.path.append('../')

import torch
from utils.utils import evaluate,train,prepare
from models.SFI_FIM import SFIModel_pipeline,SFIModel_gating,SFIModel_unified

In [2]:
hparams = {
    'scale':'demo',
    'name':'sfi-fim',
    'batch_size':5,
    'title_size':20,
    'his_size':50,
    'npratio':4,
    'dropout_p':0.2,
    'embedding_dim':300,
    'filter_num':150,
    'epochs':1,
    'metrics':'group_auc,mean_mrr,ndcg@5,ndcg@10',
    'device':'cuda:1',
    'attrs': ['title'],
    'integration':'harmony',
    'k':10,
    'save_step':0,
    'save_each_epoch':True,
    'train_embedding':True
}

device = torch.device(hparams['device'])

In [3]:
vocab, loader_train, loader_test, loader_validate = prepare(hparams, validate=True)

In [4]:
hparams['select'] = 'gating'
sfiModel_gating = SFIModel_gating(hparams, vocab=vocab).to(device)

In [5]:
train(sfiModel_gating, hparams, loader_train, loader_test, loader_validate)

training...
epoch 1 , step 300 , loss: 1.5330: : 400it [00:15, 26.17it/s]
0it [00:00, ?it/s]saved model of epoch 1
testing...
3623it [00:34, 105.45it/s]
0it [00:00, ?it/s]impression 1 has been stripped because of lacking 1 label
impression 2 has been stripped because of lacking 1 label
impression 3 has been stripped because of lacking 1 label
impression 4 has been stripped because of lacking 1 label
impression 5 has been stripped because of lacking 1 label
impression 6 has been stripped because of lacking 1 label
impression 7 has been stripped because of lacking 1 label
impression 8 has been stripped because of lacking 1 label
impression 9 has been stripped because of lacking 1 label
impression 10 has been stripped because of lacking 1 label
impression 11 has been stripped because of lacking 1 label
impression 12 has been stripped because of lacking 1 label
impression 13 has been stripped because of lacking 1 label
impression 14 has been stripped because of lacking 1 label
impression 1

SFIModel_gating(
  (softmax): Softmax(dim=-1)
  (CNN_d1): Conv1d(300, 150, kernel_size=(3,), stride=(1,), padding=(1,))
  (CNN_d2): Conv1d(300, 150, kernel_size=(3,), stride=(1,), padding=(2,), dilation=(2,))
  (CNN_d3): Conv1d(300, 150, kernel_size=(3,), stride=(1,), padding=(3,), dilation=(3,))
  (ReLU): ReLU()
  (LayerNorm): LayerNorm((150,), eps=1e-05, elementwise_affine=True)
  (DropOut): Dropout(p=0.2, inplace=False)
  (SeqCNN3D): Sequential(
    (0): Conv3d(3, 32, kernel_size=[3, 3, 3], stride=(1, 1, 1), padding=(1, 1, 1))
    (1): ReLU()
    (2): MaxPool3d(kernel_size=[3, 3, 3], stride=[3, 3, 3], padding=0, dilation=1, ceil_mode=False)
    (3): Conv3d(32, 16, kernel_size=[3, 3, 3], stride=(1, 1, 1), padding=(1, 1, 1))
    (4): ReLU()
    (5): MaxPool3d(kernel_size=[3, 3, 3], stride=[3, 3, 3], padding=0, dilation=1, ceil_mode=False)
  )
  (learningToRank): Linear(in_features=64, out_features=1, bias=True)
)

In [ ]:
hparams['select'] = 'pipeline'
sfiModel_pipeline = SFIModel_pipeline(hparams, vocab=vocab).to(device)

In [ ]:
train(sfiModel_pipeline, hparams, loader_train, loader_test, loader_validate)

In [5]:
hparams['select'] = 'unified'
sfiModel_unified = SFIModel_unified(hparams, vocab=vocab).to(device)

In [ ]:
train(sfiModel_unified, hparams, loader_train, loader_test, loader_validate)

In [ ]:
hparams['select'] = 'unified'
hparams['integration'] = 'gate'
sfiModel_unified = SFIModel_unified(hparams, vocab=vocab).to(device)

In [ ]:
train(sfiModel_unified, hparams, loader_train, loader_test, loader_validate)

In [9]:
t = SFIModel_gating(hparams, vocab).to(device)
t.cdd_size = 1

record = next(iter(loader_test))